# Data Handling and CSV Creating

In [1]:
import glob
import re
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from pywt import wavedec
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import scipy.integrate as integrate
from statsmodels.tsa.ar_model import AutoReg
from pylab import figure,clf,plot,xlabel,ylabel,title,grid,axes,show
from scipy.signal import find_peaks
import scipy.signal
import pickle
from scipy import signal
from scipy.signal import butter, filtfilt

In [2]:
# Dataset Classes:
# ________________

# Yukari-->Up
# Asagi-->Down
# Sag-->Right 
# Sol-->Left
# Kirp-->Blink 
# Center ????
# what h and v reprenets ??? 


In [3]:
class_names = {
    0: 'Up',
    1: 'Down',
    2: 'Right',
    3: 'Left',
    4: 'Blink'
}

In [4]:
names= np.arange(0,503)

In [5]:
df = pd.read_csv('Data Handling Concat.csv', names=names)

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,500,501,502
0,123,122,127,130,123,123,127,128,125,125,...,143,139,143,141,139,138,142,136,137,1
1,120,132,120,127,126,125,120,125,122,118,...,143,140,139,144,142,139,139,143,134,1
2,136,130,141,136,137,135,144,133,143,144,...,143,141,148,148,144,149,147,150,150,1
3,148,150,139,144,144,144,139,148,142,146,...,135,128,131,132,132,131,133,135,136,1
4,118,112,121,120,116,115,119,116,118,115,...,143,138,135,144,133,133,137,140,95,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 503 entries, 0 to 502
dtypes: int64(503)
memory usage: 393.1 KB


In [8]:
x=df.iloc[0:15,:-1]
x

,0,1,2,3,4,5,6,7,8,9,...,492,493,494,495,496,497,498,499,500,501
0,123,122,127,130,123,123,127,128,125,125,...,149,143,139,143,141,139,138,142,136,137
1,120,132,120,127,126,125,120,125,122,118,...,135,143,140,139,144,142,139,139,143,134
2,136,130,141,136,137,135,144,133,143,144,...,152,143,141,148,148,144,149,147,150,150
3,148,150,139,144,144,144,139,148,142,146,...,135,135,128,131,132,132,131,133,135,136
4,118,112,121,120,116,115,119,116,118,115,...,136,143,138,135,144,133,133,137,140,95
5,125,132,131,125,131,128,129,128,133,136,...,143,139,139,142,141,135,143,139,141,149
6,121,121,117,123,122,119,121,124,124,118,...,130,136,137,137,130,135,139,132,137,140
7,131,129,123,135,121,126,132,124,131,132,...,137,136,135,139,137,138,131,137,139,146
8,128,129,132,129,131,135,124,131,134,120,...,133,138,139,132,132,139,136,133,132,144
9,131,128,125,127,123,125,133,127,126,124,...,142,138,139,139,132,134,137,137,133,132


In [9]:
y=df.iloc[0:1,-1:]
y

,502
0,1


In [10]:
def butter_bandpass_filter(Input_Signal, Low_Cutoff, High_Cutoff,Sampling_Rate, order):
    nyq = 0.5*Sampling_Rate
    low=Low_Cutoff/nyq
    high =High_Cutoff/nyq
    Numerator, denominator = butter(order,[low,high],btype='band', output='ba',analog=False, fs=None)
    filtered = filtfilt(Numerator,denominator,Input_Signal)
    return filtered

In [11]:
def prepare(x):
    # Preprocessing
    filtered_signal = butter_bandpass_filter(x,Low_Cutoff=0.5,High_Cutoff=30.0,Sampling_Rate=176,order=2)
    resampled_Signal = []
    for i in filtered_signal:
        re_Sgnl = signal.resample(i,60)
        resampled_Signal.append(re_Sgnl)
    Mean=[np.mean(resampled_Signal[i]) for i in range(len(resampled_Signal))]
    RemovedDC_signal=[(resampled_Signal[i]-Mean[i]) for i in range (len(resampled_Signal))]
    # Extract features
    coeffs = wavedec(RemovedDC_signal , 'db4', level = 2)
    res = pywt.waverec([coeffs[0],coeffs[1]],'db4')
    scaler=pickle.load(open('scaler1.pkl', 'rb'))
    X_test=scaler.transform(res)  
    abc = pickle.load(open('svm.pkl', 'rb'))
    ytest_pred_abc=abc.predict(X_test)
    return ytest_pred_abc


In [12]:
prediction=prepare(x)
prediction


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)